## Extract pixel values in Google Earth Engine (GEE) for Fluxnet sites

In [1]:
!pip install earthengine-api

For more information about the installation of Google Earth Engine (Python Api) , please visit https://developers.google.com/earth-engine/guides/python_install

In [2]:
import os
import ee
import pandas as pd

Authenticate to the Earth Engine servers:

In [3]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=YSd0uITA2nwV3denlJOIFMMHYMGv3Zt-Txe7aBsNRjQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AfDhmrjxg2EyoIwvBCwd6rwvxLzcrNM0J7TfFasIhn-NH5j2CB6ntO3T13w

Successfully saved authorization token.


Initialize the API:

In [4]:
ee.Initialize()

In [5]:
def func_feat(feature,index):
    """ Orgnazition of output csv

    """
    coordinates = feature.geometry().coordinates()
    feature = feature.set("lon", coordinates.get(0))
    feature = feature.set("lat", coordinates.get(1))
    feature = feature.set("imageId", index)
    feature = feature.set("pointId", feature.get("system:index"))
    return feature

def func_cov(image,points):
    """ get mean value in the area defined by scale

    """
    scale = 500
    fCol = image.reduceRegions(collection = points,
                  reducer = ee.Reducer.mean(),
                  scale = scale)
    index= image.get("system:index")
    fCol = fCol.map(lambda x: func_feat(x,index))
    return fCol

You need to upload the [locations of Fluxnet2015 sites](https://fluxnet.org/sites/site-list-and-pages/) in shape files format to your GEE assets **or** open this [fluxnet2015](https://code.earthengine.google.com/?asset=users/mrhuaize/fluxnet_sites15)(or [link](https://code.earthengine.google.com/?asset=users/mrhuaize/fluxnet_sites) for [all sites](https://daac.ornl.gov/FLUXNET/guides/Fluxnet_site_DB.html)) to copy it to your assets directly. 

In [ ]:
dataset = "MODIS/006/MCD43A4" #check details and more products on Earth Engine Data Catalog
loc_of_fluxnet_towers = 'users/mrhuaize/fluxnet_sites15' 
start_time = "2017-1-1"
end_time = "2017-1-5"# Notice that GEE up to 5000 records can be extracted at on time
band = ['Nadir_Reflectance_Band1','Nadir_Reflectance_Band2','Nadir_Reflectance_Band3','Nadir_Reflectance_Band4','Nadir_Reflectance_Band5','Nadir_Reflectance_Band5','Nadir_Reflectance_Band7'] # check details especially the scale factor on Earth Engine Data Catalog 

In [6]:
dataset = "MODIS/006/MCD43A4" #check details and more products on Earth Engine Data Catalog
loc_of_fluxnet_towers = 'users/mrhuaize/fluxnet_sites15' 
start_time = pd.to_datetime("2000-01-01")
end_time = start_time + pd.Timedelta('5D')# Notice that GEE up to 5000 records can be extracted at on time
bands = ['Nadir_Reflectance_Band1','Nadir_Reflectance_Band2','Nadir_Reflectance_Band3','Nadir_Reflectance_Band4','Nadir_Reflectance_Band5','Nadir_Reflectance_Band6','Nadir_Reflectance_Band7'] # check details especially the scale factor on Earth Engine Data Catalog 

In [7]:
l8 = ee.ImageCollection(dataset)
points = ee.FeatureCollection(loc_of_fluxnet_towers)

In [13]:
start_time.strftime('%Y-%m-%d')

'2000-01-11'

In [17]:
scol = l8.filterDate(start_time.strftime('%Y-%m-%d'),end_time.strftime('%Y-%m-%d')).select('Nadir_Reflectance_Band1')
scol = scol.map(lambda img: func_cov(img,points)).flatten()
task = ee.batch.Export.table.toDrive(
  collection = scol,
  folder = 'Reflectance',
  description = "fluxnet_",
  fileFormat = "CSV",
  selectors =  ["imageId", "pointId", "lon", "lat", "mean"]
)
task.start()

In [14]:
# Export the table to your Google Drive
while start_time < pd.to_datetime('2016-01-01'):
  scol = l8.filterDate(start_time.strftime('%Y-%m-%d'),end_time.strftime('%Y-%m-%d')).select('Nadir_Reflectance_Band1')
  scol = scol.map(lambda img: func_cov(img,points)).flatten() 
  task = ee.batch.Export.table.toDrive(
      folder = 'Reflectance',
      collection = scol,
      description = start_time.strftime('%Y-%m-%d'),
      fileFormat = "CSV",
      # selectors =  ["imageId", "SITE_ID", "lon", "lat"] + bands
  )

  task.start()
  print(str(str(start_time) + dataset + ' is done!'))
  start_time = end_time
  end_time = start_time + pd.Timedelta('5D')# Notice that GEE up to 5000 records can be extracted at on time

2000-01-11 00:00:00MODIS/006/MCD43A4 is done!
2000-01-16 00:00:00MODIS/006/MCD43A4 is done!


KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
from datetime import datetime
# Post-processing if you interest
def time2Index(x):
  """ Dealing with timestamp

  """
  t = []
  ts  = x.astype(np.str).values
  for time in ts:
      t.append(datetime.strptime(time,'%Y_%m_%d'))
  return t

def getGEEvar(file,vari,scale,time = '30min'):
  """ Scaling and interpolate

  """
  LAI = pd.read_csv(file)
  LAI.imageId = pd.DatetimeIndex(time2Index(LAI.imageId)).round('min')
  LAI.columns = LAI.columns.str.replace('imageId','TIMESTAMP')
  LAI.columns = LAI.columns.str.replace('mean',vari)
  LAI.set_index('TIMESTAMP',inplace = True)
  LAI = LAI.resample(time).interpolate()
  LAI = LAI.drop(['lon','lat','pointId'],axis = 1)#.resample(time).interpolate()
  LAI[vari] = LAI[vari]*scale
  return LAI